#### Basic Data Cleaning
Basic data cleaning steps like lowering of case, transforming of dates are done here

In [5]:
import pandas as pd
import numpy as np
import re as re
import nltk
import string, unicodedata
from nltk.stem.porter import *
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from datetime import datetime, timedelta
import itertools
from spellchecker import SpellChecker
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

spell = SpellChecker()


data_file = "Data/all_data_raw_final.csv"
data = pd.read_csv(data_file)
data.columns = data.columns.str.strip().str.replace(" ","_") # reason for doing this is cause cannot get columns with space into a list

#lower case
data['Review'] = [word.lower() for word in data['Review']]

# #Strip off the Variation Word
# data['Product_Purchase'] = data['Product_Purchase'].replace('Variation', '')

#Put all columns into a list for processing
reviews_list = data['Review'].tolist()
platform_list = data['Platform'].tolist()
brand_list = data['Brand'].tolist()
category_list = data['Category'].tolist()
product_list = data['Product_Name'].tolist()
price_list = data['Price'].tolist()
reviewer_list = data['Reviewer'].tolist()
review_list = data['Review'].tolist()
purchase_list = data['Product_Purchase'].tolist()
ratings_list = data['Ratings'].tolist()
response_list = data['Response'].tolist()
date_review_list = data["Date_Of_Review"].tolist()


new_date =[]


for i in range(len(brand_list)):  
    date_review = date_review_list[i]
    
    #Change the date to the scrape date accordingly
    scrape_date = datetime(2020, 3, 23)
    
    if platform_list[i] == 'Lazada':
        if 'days ago' in date_review_list or 'day ago' in date_review_list :
            day = date_review.replace('days ago', '').replace('day ago', '')
            date_review = datetime.today() - timedelta(days=int(day))
            date_review = scrape_date - timedelta(days=int(day))
            date_review = date_review.strftime("%d-%m-%y")

        elif 'weeks ago' in date_review_list or 'week ago' in date_review_list:
            day = date_review.replace('weeks ago', '').replace('week ago', '')
            day = int(day)*7
            date_review = datetime.today() - timedelta(days=day)
            date_review = scrape_date - timedelta(days=day)
            date_review = date_review.strftime("%d-%m-%y")


        elif 'hours ago' in date_review_list or 'hour ago' in date_review_list:
            hours = date_review.replace('hours ago', '').replace('hour ago', '')
            date_review = scrape_date - timedelta(hours=int(hours))
            date_review = date_review.strftime("%d-%m-%y")

        new_date.append(date_review)
    else:
        new_date.append(date_review)
        
data["Date_Of_Review"] = new_date

new_date_review_list = data["Date_Of_Review"].tolist()

#for ID to group the same review tgt
num_list = []

for i in range(len(data)):
    num_list.append(i)
    
    
data['ID'] = num_list

#### Emoji Processing
In this section, we seek to identify the sentiments of the emoji. Each emoji have its own sentiment and the emoji sentiment is labelled via http://kt.ijs.si/data/Emoji_sentiment_ranking/index.html. It consists of emoji sentiments taken from twitter and labelled by human experts. It outputs "Emoji_Sentiment_Data_v1.0.csv'.

In [6]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)
import re
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import spacy
import demoji

#emoji_sentiment_ranking_file is output fromhttp://kt.ijs.si/data/Emoji_sentiment_ranking/index.html
emoji_sentiment_ranking_file = "Emoji_Sentiment_Data_v1.0.csv"
emoji_sentiment_ranking = pd.read_csv(emoji_sentiment_ranking_file)
emoji_sentiment_ranking.columns = emoji_sentiment_ranking.columns.str.strip().str.replace(" ","_")

#import all the useful columns to a list for easy usage later on
emoji_unicode = emoji_sentiment_ranking['Unicode_codepoint'].tolist()
occurences = emoji_sentiment_ranking['Occurrences'].tolist()
positive = emoji_sentiment_ranking['Positive'].tolist()
neutral = emoji_sentiment_ranking['Neutral'].tolist()
negative = emoji_sentiment_ranking['Negative'].tolist()

#takes in a string of emoji and return the integer value of polarity.
def getEmojiSentiment(emoji_string): #eg. emoji_string = "😀😁😊👍🏻"
    positive_sentiment = 0
    neutral_sentiment = 0
    negative_sentiment = 0
    emoji_splitted = emoji_string.split(" ") #split each emoji as one emoji
    
    
    for i in range(len(emoji_splitted)): #for each emoji, do this
        current_emoji = emoji_splitted[i] 
        current_emoji = current_emoji[0] #some emoji is stored as length 2, one for emoji one for color. eg. test_string = "👍🏻" which is length 2. test_string[0] will return 👍

        
        if len(current_emoji) == 1:
            unicode = '0x{:X}'.format(ord(current_emoji)).lower() #get the particular emoji's unicode
            
            if unicode in emoji_unicode: #find the emoji's unicode from the list of unicode provided in the web
                find_unicode_index = emoji_unicode.index(unicode)
                
                #find the general sentiment of each emoji from the web
                if int(positive[find_unicode_index]) >= int(neutral[find_unicode_index]) and int(positive[find_unicode_index]) >= int(negative[find_unicode_index]):
                        positive_sentiment += 1
                elif int(negative[find_unicode_index]) >= int(neutral[find_unicode_index]) and int(negative[find_unicode_index]) >= int(positive[find_unicode_index]):
                        negative_sentiment += 1
                elif int(neutral[find_unicode_index]) >= int(negative[find_unicode_index]) and int(neutral[find_unicode_index]) >= int(positive[find_unicode_index]):
                        neutral_sentiment += 1
        else: 
            pass

    #find the sentiment of the whole emoji_string
    if positive_sentiment > neutral_sentiment and positive_sentiment > negative_sentiment:
        return ("1")
    elif negative_sentiment > neutral_sentiment and negative_sentiment > positive_sentiment:
        return ("-1")
    else:
        return ("0")

def lemmatization(texts):
    """https://spacy.io/api/annotation"""
    doc = sp(texts)
    
    texts_out = ([token.lemma_ for token in doc])
    return texts_out

#upload the appos
from word_dict import appos

def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

### Splitting of text review and data cleaning
Reviews are splitted using the sent_tokenize function in nltk and some data cleaning process, extracting of emojis, removing of special characters, transforming negations reducing word length (for words like "ggooooooddd" and spelling corrections are all done here 

In [7]:
import re
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import spacy
import demoji
#upload the appos
from word_dict import appos

stop_list = stopwords.words('english')
# Porter Stemming
stemmer = PorterStemmer()

num = []
reviews = []
platform = []
brand = []
category = []
product = []
price = []
reviewer = []
review = []
purchase = []
ratings = []
date = []
response = []
splitted_reviews = []
emoji_polarity = []

for i in range(len(reviews_list)):
    
    #Splitting of review into different sentences
    review_splitted = sent_tokenize(review_list[i])
    

    for splitted in review_splitted:
        
        #Extracting of emojis and giving them their own sentiments and apeending it into a list
        emoji_dictionary = demoji.findall(splitted)
        if len(emoji_dictionary) == 0:
            emoji_sentiment=""
        else:
            temp_list = ""
            for key in emoji_dictionary:
                temp_list += key + " "
            temp_list = temp_list[0:-1]
            emoji_extract= temp_list
            emoji_sentiment = getEmojiSentiment(emoji_extract)
        
        #transforming negations like 'don't' to 'do not'
        processed_review = splitted.split()
        processed_review =[appos[w] if w in appos else w for w in processed_review]
        processed_review = " ".join(processed_review) 

        processed_review = processed_review.strip()
        
        # Remove all the special characters
        processed_review = re.sub(r'\W', ' ', processed_review)
        # Removing prefixed 'b'
        processed_review = re.sub(r'^b\s+', '', processed_review)
        # remove all single characters contains a white space character
        processed_review= re.sub(r'^[a-zA-Z]$', ' ', processed_review)
        # Substituting multiple spaces with single space
        processed_review = re.sub(r'\s+', ' ', processed_review, flags=re.I)
        
        processed = processed_review.strip()

        # final check if the processed_review is empty
        final = []
        if processed != "":
            tokenized = word_tokenize(processed)
            #removed additional letters eg "gooooooodddddd" to "good"
            for w in tokenized:
                processed_2 = reduce_lengthening(w)
                # spelling correction
                processed_2 = spell.correction(processed_2)
                final.append(processed_2)
            final =' '.join(final)
#             print(final)
            num.append(num_list[i])
            reviews.append(review_list[i])  
            splitted_reviews.append(final)
            platform.append(platform_list[i])
            brand.append(brand_list[i])
            category.append(category_list[i])
            price.append(price_list[i])
            product.append(product_list[i])
            reviewer.append(reviewer_list[i])
            review.append(review_list[i])
            purchase.append(purchase_list[i])
            ratings.append(ratings_list[i])
            date.append(new_date_review_list[i])
            response.append(response_list[i])
            emoji_polarity.append(emoji_sentiment)
              

#### Analysis
All the models created for topic classication and sentiment classification have been saved to a pickle and loaded here. All the different data preprocessing steps have been included and the models are then applied to get the topics and sentiment.

In [9]:
# ========================================================================================== #  
# IMPORT PICKLE AND LOAD SAVE CLASSIFIER PICKLE TO DO THE CLASSIFICATION
# ========================================================================================== #  

import pickle
#load topic classifier
classifier_saved = open("topic_classifier.pickle", "rb") #binary read
topic_classifier = pickle.load(classifier_saved)
classifier_saved.close()

#load topic vectorizer
vectorizer_saved = open("topic_vectorizer.pickle", "rb") #binary read
topic_vectorizer = pickle.load(vectorizer_saved)
vectorizer_saved.close()

#load sentiment classifier
classifier_saved = open("sentiment_classifier.pickle", "rb") #binary read
sentiment_classifier = pickle.load(classifier_saved)
classifier_saved.close()

#load sentiment vectorizer
vectorizer_saved = open("sentiment_vectorizer.pickle", "rb") #binary read
sentiment_vectorizer = pickle.load(vectorizer_saved)
vectorizer_saved.close()

# ========================================================================================== #  
# APPLY TOPIC CLASSIFIER MODEL
# ========================================================================================== #   

topic = []

review_tokenized_topic = [word_tokenize(each_review) for each_review in splitted_reviews]
review_lower_topic = [[each_word.lower() for each_word in each_review] for each_review in review_tokenized_topic]
review_alpha_topic = [[each_word for each_word in each_review if re.search('^[a-z]+$', each_word)] for each_review in review_lower_topic]
review_stop_topic = [[each_word for each_word in each_review if each_word not in stop_list] for each_review in review_alpha_topic]
review_stem_topic = [[stemmer.stem(each_word) for each_word in each_review] for each_review in review_stop_topic]


all_data_cleaned_topic = []
for each_sentence in review_stem_topic:
#         print("each",each_sentence)
    sentence = ""
    for each_word in each_sentence:
        sentence += each_word + " "
    sentence = sentence[0:-1]   
#         print(sentence)  
    #append to put in the model
    all_data_cleaned_topic.append(sentence)
print(len(all_data_cleaned_topic))

all_data_cleaned_vec_topic = topic_vectorizer.transform(all_data_cleaned_topic)
labels = topic_classifier.predict(all_data_cleaned_vec_topic)

for label in labels:
    if label == 0:
        topic.append('delivery')
    elif label == 1:
        topic.append('service')
    elif label == 2:
        topic.append('price')
    elif label == 3:
        topic.append('quality')
    elif label == 4:
        topic.append('no_topic')
    
            
# ========================================================================================== #  
# APPLY EMOJI SENTIMENT CLASSIFIER MODEL
# ========================================================================================== #           
overall_polarity = []
sp = spacy.load('en', disable=['parser', 'ner'])
from nltk.corpus import stopwords

to_remove = ['not', 'no','but','if']
new_stopwords = set(stopwords.words('english')).difference(to_remove)
stemmer = PorterStemmer()


review_tokenized_polarity = [lemmatization(each_review) for each_review in splitted_reviews]
review_lower_polarity = [[each_word.lower() for each_word in each_review] for each_review in review_tokenized_polarity]
review_stop_polarity = [[each_word for each_word in each_review if each_word not in new_stopwords] for each_review in review_lower_polarity]
review_alpha_polarity = [[each_word for each_word in each_review if re.search('^[a-z]+$', each_word)] for each_review in review_stop_polarity]

all_data_cleaned_polarity = []
for each_sentence in review_alpha_polarity:
    sentence = ""
    for each_word in each_sentence:
        sentence += each_word + " "
    sentence = sentence[0:-1]   

    #append to put in the model
    all_data_cleaned_polarity.append(sentence)

#     print(all_data_cleaned)

all_data_cleaned_vec_polarity = sentiment_vectorizer.transform(all_data_cleaned_polarity)
polarity = sentiment_classifier.predict(all_data_cleaned_vec_polarity)

#compare emoji polarity and text polarity

for i in range(len(polarity)):

    if emoji_polarity[i] == '-1' and polarity[i] > -1:
        overall_polarity.append('-1')
    elif polarity[i] == 0:
        overall_polarity.append("0")
    elif polarity[i] == 1:
        overall_polarity.append('1')
    elif polarity[i] == -1:
        overall_polarity.append("-1")
    elif emoji_polarity[i] == '0':
        overall_polarity.append("0")
    elif emoji_polarity[i] == '1':
        overall_polarity.append('1')
    elif emoji_polarity[i] == '-1':
        overall_polarity.append("-1")

new_data = {'ID':num, 'Platform':platform, 'Brand': brand, 'Category': category, "Product_Name": product, "Price": price, "Reviewer": reviewer,"Review": reviews, "Splitted_review":splitted_reviews, "Product_Purchase":purchase, "Ratings":ratings,"Date_Of_Review":date,"Response":response,"topic":topic,"Overall_polarity":polarity}
new_df = pd.DataFrame.from_dict(new_data)
new_df.to_csv("Final_Final_data.csv")

18581


### Commbining of the topic for overall polarity
for reviews that mention the same topic in 2 different sentences, the topic and sentiment will be combined together

In [10]:
from statistics import mode

#find the mode of each review
result = new_df.groupby(['ID','topic'])['Overall_polarity'].agg(lambda x:x.value_counts().index[0])
 
review_mode = pd.DataFrame(result)
review_mode.reset_index(inplace=True) 
review_mode.head()

,ID,topic,Overall_polarity
0,0,delivery,1
1,0,price,1
2,1,delivery,1
3,2,delivery,1
4,2,quality,0


In [11]:
# Combined the raw cleaned file with the
final = pd.merge (data,review_mode, how='inner', on=['ID'])
final
# final.to_csv('All_Data_Dashboard_NewFinal.csv')

,Platform,Brand,Category,Product_Name,Price,Reviewer,Review,Product_Purchase,Ratings,Date_Of_Review,Response,ID,topic,Overall_polarity
0,Shopee,L'Oreal Paris,Serum & Treatment,"L'Oreal Paris Revitalift Crystal Micro-Essence Water, 130ml",35.9,samyeez,item received in good condition. good deal! bought during flash deal.,NaN,5.0,18/2/2020 9:30,no,0,delivery,1
1,Shopee,L'Oreal Paris,Serum & Treatment,"L'Oreal Paris Revitalift Crystal Micro-Essence Water, 130ml",35.9,samyeez,item received in good condition. good deal! bought during flash deal.,NaN,5.0,18/2/2020 9:30,no,0,price,1
2,Shopee,L'Oreal Paris,Serum & Treatment,"L'Oreal Paris Revitalift Crystal Micro-Essence Water, 130ml",35.9,chongesther72,item received in good condition,NaN,4.0,14/2/2020 21:28,no,1,delivery,1
3,Shopee,L'Oreal Paris,Serum & Treatment,"L'Oreal Paris Revitalift Crystal Micro-Essence Water, 130ml",35.9,aprilkaye1,super fast delivery. will surely order more in future 😊,NaN,5.0,15/2/2020 0:16,no,2,delivery,1
4,Shopee,L'Oreal Paris,Serum & Treatment,"L'Oreal Paris Revitalift Crystal Micro-Essence Water, 130ml",35.9,aprilkaye1,super fast delivery. will surely order more in future 😊,NaN,5.0,15/2/2020 0:16,no,2,quality,0
5,Shopee,L'Oreal Paris,Serum & Treatment,"L'Oreal Paris Revitalift Crystal Micro-Essence Water, 130ml",35.9,iceszmanjasz,nice smell...,NaN,5.0,15/2/2020 0:32,no,3,quality,0
6,Shopee,L'Oreal Paris,Serum & Treatment,"L'Oreal Paris Revitalift Crystal Micro-Essence Water, 130ml",35.9,zenheng,"delivery was quite fast, and item was on discount too so i couldn’t complain. so far liking this product! will continue to use it :)",NaN,4.0,6/2/2020 23:58,no,4,delivery,1
7,Shopee,L'Oreal Paris,Serum & Treatment,"L'Oreal Paris Revitalift Crystal Micro-Essence Water, 130ml",35.9,zenheng,"delivery was quite fast, and item was on discount too so i couldn’t complain. so far liking this product! will continue to use it :)",NaN,4.0,6/2/2020 23:58,no,4,quality,1
8,Shopee,L'Oreal Paris,Serum & Treatment,"L'Oreal Paris Revitalift Crystal Micro-Essence Water, 130ml",35.9,glennoeestaquio,received item sealed properly.\r\r\nquick delivery too.\r\r\nitem not tested yet as for gifting purpose!,NaN,5.0,6/2/2020 12:44,no,5,delivery,1
9,Shopee,L'Oreal Paris,Serum & Treatment,"L'Oreal Paris Revitalift Crystal Micro-Essence Water, 130ml",35.9,glennoeestaquio,received item sealed properly.\r\r\nquick delivery too.\r\r\nitem not tested yet as for gifting purpose!,NaN,5.0,6/2/2020 12:44,no,5,quality,0
